# Linear Solver Performance: XDG Poisson, Single Core

### Part 1, Benchmark Setup and Execution

In [ ]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

Execution Date/time is 11/08/2021 11:47:30


In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 4
Count: 1



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\Users\flori\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 162
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\Users\flori\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#22.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
using BoSSS.Application.XdgPoisson3;

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,Password,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases == , C:\Users\flori == ]",,,,,,,,,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\dc1\userspace\kummer\cluster,False,FDY-WindowsHPC,dotnet,,[ \\dc1\userspace\kummer == ],FDY\kummer,DC2,<null>,Normal,True,,,,,,,
2,BoSSS.Application.BoSSSpad.SlurmClient,W:\bosss_deploy,False,LbI,dotnet,,,fk69umer,lcluster8.hrz.tu-darmstadt.de,,,,<null>,C:\Users\flori\.ssh\id_rsa,<null>,/home/fk69umer/bosss_deploy,project01287,<null>,False
3,BoSSS.Application.BoSSSpad.SlurmClient,X:\bosss_deploy,False,Lb2-specialPrj,dotnet,,[ X: == /work/scratch/fk69umer ],fk69umer,lcluster16.hrz.tu-darmstadt.de,,,,<null>,C:\Users\flori\.ssh\id_rsa,"[ #SBATCH -p test24, #SBATCH -C avx512, #SBATCH --mem-per-cpu=8000 ]",/work/scratch/fk69umer/bosss_deploy,special00006,<null>,False


In [ ]:
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\BoSSS-Databases == , C:\Users\flori == ]"


In [ ]:
wmg.Init("LinslvPerf_XdgPoissonSer");
wmg.AllJobs

Project name is set to 'LinslvPerf_XdgPoissonSer'.
Opening existing database 'C:\BoSSS-Databases\LinslvPerf_XdgPoissonSer'.


## Utility definitions

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }*/
}

## Init grids and save to database

In [ ]:
wmg.Grids

Opening existing database '\\dc1\userspace\kummer\LinslvPerf_XdgPoissonSer'.
DirectoryNotFoundException caught during creation/opening of database: Could not find a part of the path 'X:\LinslvPerf_XdgPoissonSer'..


#0: { Guid = e526799b-bc50-40f5-9e07-393469db64e5; Name = LinslvPerf_XdgPoissonSer_J262144; Cell Count = 262144; Dim = 3 }
#1: { Guid = 9574aa7b-797e-4e97-896e-745b18dba0cb; Name = LinslvPerf_XdgPoissonSer_J32768; Cell Count = 32768; Dim = 3 }
#2: { Guid = 56dfb2c4-8fe4-40a4-bba4-4496f760b623; Name = LinslvPerf_XdgPoissonSer_J4096; Cell Count = 4096; Dim = 3 }
#3: { Guid = 04bbd46f-eb85-45c8-bfa6-218818e2bf2e; Name = LinslvPerf_XdgPoissonSer_J512; Cell Count = 512; Dim = 3 }
#4: { Guid = 24e5a297-1907-4f13-b7be-efb1407f8632; Name = LinslvPerf_XdgPoissonSer_J64; Cell Count = 64; Dim = 3 }
#5: { Guid = d3faf6e4-b4e0-4a52-bdda-606cecaee32a; Name = LinslvPerf_XdgPoissonSer_J8; Cell Count = 8; Dim = 3 }


Create meshes in various resolutions:
- domain $\Omega = (-1,1)^3$; 
- a Dirichlet boundary is set everywhere; the challenge of this benchmark,
  from a numerical point, is the 1:1000 ratio in the diffusion coefficient

In [ ]:
int[] Resolutions_3D = new int[] { 2, 4, 8, 16, 32, 64 };
//int[] Resolutions_3D = new int[] { 2 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] zNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "_J" + J);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(string.Format("_J"+ J))); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            return "Dirichlet";
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Found Grid: { Guid = d3faf6e4-b4e0-4a52-bdda-606cecaee32a; Name = LinslvPerf_XdgPoissonSer_J8; Cell Count = 8; Dim = 3 }
Found Grid: { Guid = 24e5a297-1907-4f13-b7be-efb1407f8632; Name = LinslvPerf_XdgPoissonSer_J64; Cell Count = 64; Dim = 3 }
Found Grid: { Guid = 04bbd46f-eb85-45c8-bfa6-218818e2bf2e; Name = LinslvPerf_XdgPoissonSer_J512; Cell Count = 512; Dim = 3 }
Found Grid: { Guid = 56dfb2c4-8fe4-40a4-bba4-4496f760b623; Name = LinslvPerf_XdgPoissonSer_J4096; Cell Count = 4096; Dim = 3 }
Found Grid: { Guid = 9574aa7b-797e-4e97-896e-745b18dba0cb; Name = LinslvPerf_XdgPoissonSer_J32768; Cell Count = 32768; Dim = 3 }
Found Grid: { Guid = e526799b-bc50-40f5-9e07-393469db64e5; Name = LinslvPerf_XdgPoissonSer_J262144; Cell Count = 262144; Dim = 3 }


In [ ]:
grids

#0: { Guid = d3faf6e4-b4e0-4a52-bdda-606cecaee32a; Name = LinslvPerf_XdgPoissonSer_J8; Cell Count = 8; Dim = 3 }
#1: { Guid = 24e5a297-1907-4f13-b7be-efb1407f8632; Name = LinslvPerf_XdgPoissonSer_J64; Cell Count = 64; Dim = 3 }
#2: { Guid = 04bbd46f-eb85-45c8-bfa6-218818e2bf2e; Name = LinslvPerf_XdgPoissonSer_J512; Cell Count = 512; Dim = 3 }
#3: { Guid = 56dfb2c4-8fe4-40a4-bba4-4496f760b623; Name = LinslvPerf_XdgPoissonSer_J4096; Cell Count = 4096; Dim = 3 }
#4: { Guid = 9574aa7b-797e-4e97-896e-745b18dba0cb; Name = LinslvPerf_XdgPoissonSer_J32768; Cell Count = 32768; Dim = 3 }
#5: { Guid = e526799b-bc50-40f5-9e07-393469db64e5; Name = LinslvPerf_XdgPoissonSer_J262144; Cell Count = 262144; Dim = 3 }


## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [ ]:
/*static class InitialValues {
    public static double RHS(double[] X) {
        return -Math.Sin(X[0]);
    }
    
    public static double DirichletBC(double[] X) {
        return 0.0;
    }
    
    public static double NeumannBC(double[] X) {
       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8)
           return 0;
       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8))
           return 0;

       return Math.Cos(10.0);
   }
}*/

### Setup of Parameter Study

In [ ]:
List<XdgPoisson3Control> controls = new List<XdgPoisson3Control>();

Polynomial degrees to test:

In [ ]:
// polynomial degrees to test
//int[] PolyDegS = new int[] {2, 3, 4, 5, 6};
int[] PolyDegS = new int[] {2, 3, 5};

Solvers which we want to instrument:

In [ ]:
using BoSSS.Solution.AdvancedSolvers;

In [ ]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.classic_pardiso, 
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.exp_gmres_levelpmg,
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [ ]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.classic_pardiso:
        case LinearSolverCode.classic_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 5000000; // Up to 5 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [ ]:
List<XdgPoisson3Control> controls = new List<XdgPoisson3Control>();
controls.Clear();
foreach(LinearSolverCode solver_name in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {
    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J*Np > GetMaxAllowedDOF(solver_name))
        continue;
    
    var ctrl = new XdgPoisson3Control();
    controls.Add(ctrl);

    string caseName = string.Format("XdgPoisson-J{0}_k{1}_{2}", J, k, solver_name);
    Console.WriteLine("setting up: " + caseName);
    ctrl.SessionName = caseName;
    
    ctrl.SetGrid(grd);
    ctrl.savetodb = true; //for debug's sake
    ctrl.SetDGdegree(k);
    
    ctrl.LinearSolver.SolverCode           = solver_name;
    ctrl.LinearSolver.NoOfMultigridLevels  = int.MaxValue;
    ctrl.LinearSolver.ConvergenceCriterion = 1e-8;
    
    double radius           = 0.7;
    ctrl.ExcactSolSupported = false;
    ctrl.InitialValues.Add("Phi", new Formula("X => X[0].Pow2()+X[1].Pow2()+X[2].Pow2()-"+radius+".Pow2()"));
    ctrl.MU_A = -1;
    ctrl.MU_B = -1000;
    ctrl.InitialValues.Add("rhs#A", new Formula("X => 1"));
    ctrl.InitialValues.Add("rhs#B", new Formula("X => 1"));

    //ctrl.CutCellQuadratureType = XQuadFactoryHelper.MomentFittingVariants.Classic;
    //ctrl.SetDefaultDiriBndCnd  = true;
    //ctrl.ViscosityMode         = XLaplace_Interface.Mode.SIP;
        
    ctrl.AgglomerationThreshold = 0.1;
    
    controls.Add(ctrl);
}
}
}

setting up: XdgPoisson-J8_k2_classic_pardiso
setting up: XdgPoisson-J64_k2_classic_pardiso
setting up: XdgPoisson-J512_k2_classic_pardiso
setting up: XdgPoisson-J4096_k2_classic_pardiso
setting up: XdgPoisson-J32768_k2_classic_pardiso
setting up: XdgPoisson-J8_k3_classic_pardiso
setting up: XdgPoisson-J64_k3_classic_pardiso
setting up: XdgPoisson-J512_k3_classic_pardiso
setting up: XdgPoisson-J4096_k3_classic_pardiso
setting up: XdgPoisson-J32768_k3_classic_pardiso
setting up: XdgPoisson-J8_k5_classic_pardiso
setting up: XdgPoisson-J64_k5_classic_pardiso
setting up: XdgPoisson-J512_k5_classic_pardiso
setting up: XdgPoisson-J4096_k5_classic_pardiso
setting up: XdgPoisson-J8_k2_exp_Kcycle_schwarz
setting up: XdgPoisson-J64_k2_exp_Kcycle_schwarz
setting up: XdgPoisson-J512_k2_exp_Kcycle_schwarz
setting up: XdgPoisson-J4096_k2_exp_Kcycle_schwarz
setting up: XdgPoisson-J32768_k2_exp_Kcycle_schwarz
setting up: XdgPoisson-J262144_k2_exp_Kcycle_schwarz
setting up: XdgPoisson-J8_k3_exp_Kcycle_s

## Launch Jobs

In [ ]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    ctrl.RunBatch(myBatch);
}

 Submitting: XdgPoisson-J8_k2_classic_pardiso
Deploying job XdgPoisson-J8_k2_classic_pardiso ... 
Deploying executables and additional files ...
Deployment directory: C:\Users\flori\AppData\Local\BoSSS-LocalJobs\LinslvPerf_XdgPoissonSer-XdgPoisson32021Nov08_114744
copied 46 files.
   written file: control.obj
deployment finished.
Starting mini batch processor in external process...
Started mini batch processor on local machine, process id is 32084.
started.

 Submitting: XdgPoisson-J8_k2_classic_pardiso


Error: System.ArgumentException: Job with name 'XdgPoisson-J8_k2_classic_pardiso' is already defined in the workflow management.
   at BoSSS.Application.BoSSSpad.Job..ctor(String name, Type solver) in C:\Users\flori\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 63
   at BoSSS.Application.BoSSSpad.AppControlExtensions.RunBatch(AppControl ctrl, BatchProcessorClient BatchSys) in C:\Users\flori\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\AppControlExtensions.cs:line 60
   at Submission#39.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

All jobs finished.


In [ ]:
wmg.AllJobs

#0: XdgPoisson-J8_k2_classic_pardiso: FailedOrCanceled (MiniBatchProcessor client  LocalPC @C:\Users\flori\AppData\Local\BoSSS-LocalJobs)	XdgPoisson-J8_k2_classic_pardiso: FailedOrCanceled (MiniBatchProcessor client  LocalPC @C:\Users\flori\AppData\Local\BoSSS-LocalJobs)


In [ ]:
wmg.Sessions

In [ ]:
var NoSuccess = wmg.AllJobs.Values.Where(job => job.Status != JobStatus.FinishedSuccessful);
NoSuccess

#0: XdgPoisson-J8_k2_classic_pardiso: FailedOrCanceled (MiniBatchProcessor client  LocalPC @C:\Users\flori\AppData\Local\BoSSS-LocalJobs)


In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

Error: NUnit.Framework.AssertionException:   Some Jobs Failed
  Expected: 0
  But was:  1

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.Zero(Int32 actual, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Conditions.cs:line 484
   at Submission#44.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, C

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => (Si.SuccessfulTermination == false)
                                        || (Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not derminate successfully.");

List Output of some job (arbitrarily the first one):

In [ ]:
wmg.AllJobs.First().Value.Stdout

      ___           ___           ___           ___           ___     
     /\  \         /\  \         /\  \         /\  \         /\  \    
    /::\  \       /::\  \       /::\  \       /::\  \       /::\  \   
   /:/\:\  \     /:/\:\  \     /:/\ \  \     /:/\ \  \     /:/\ \  \  
  /::\~\:\__\   /:/  \:\  \   _\:\~\ \  \   _\:\~\ \  \   _\:\~\ \  \ 
 /:/\:\ \:|__| /:/__/ \:\__\ /\ \:\ \ \__\ /\ \:\ \ \__\ /\ \:\ \ \__\
 \:\~\:\/:/  / \:\  \ /:/  / \:\ \:\ \/__/ \:\ \:\ \/__/ \:\ \:\ \/__/
  \:\ \::/  /   \:\  /:/  /   \:\ \:\__\    \:\ \:\__\    \:\ \:\__\  
   \:\/:/  /     \:\/:/  /     \:\/:/  /     \:\/:/  /     \:\/:/  /  
    \::/__/       \::/  /       \::/  /       \::/  /       \::/  /   
     ~~            \/__/         \/__/         \/__/         \/__/    
                                                                      
Running with 1 MPI process (single core)
User: flori
Node: stormbreaker (rank 0)
arg #0 override from environment variable 'BOSSS_ARG_0': --control


In [ ]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail.LatestDeployment.DeploymentDirectory.Name);

LinslvPerf_XdgPoissonSer-XdgPoisson32021Nov08_114744


In [ ]:
// -------------------------------
// Additional/user-defined columns
// -------------------------------


In [ ]:
/*
static class AddCols {
    static public object SipMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("SipMatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
}*/

In [ ]:
/*WorkflowMgm.AdditionalSessionTableColums.Clear();
WorkflowMgm.AdditionalSessionTableColums.Add(
    "SipMatrixAssembly_time", AddCols.SipMatrixAssembly_time);
WorkflowMgm.AdditionalSessionTableColums.Add(
    "Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
WorkflowMgm.AdditionalSessionTableColums.Add(
    "Solver_Init_time", AddCols.Solver_Init_time);
WorkflowMgm.AdditionalSessionTableColums.Add(
    "Solver_Run_time", AddCols.Solver_Run_time);
*/